In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix

In [3]:
!pip install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0
  Attempting uninstall: cachetools
    Found existing installation: cachetools 6.2.1
    Uninstalling cachetools-6.2.1:
      Successfully uninstalled cachetools-6.2.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.5 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.

In [4]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [5]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

In [6]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

In [7]:
startup_agent = Agent(
    name="startup_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="You are agent that will greet the user and introduce to him as a tutoragent",
    instruction="""You are a friendly and professional greeting agent for a tutoring assistant system. 

**IMPORTANT: If the user's message is empty or they're just starting the conversation, immediately greet them warmly.**

Your role is to:
1. Greet the user warmly and introduce yourself as their TutorAgent assistant.
2. Briefly explain that you can help them with learning new topics, taking quizzes, and reinforcing knowledge through teach-back.
3. Ask the user what they would like to learn or do today.
4. Based on their response, acknowledge their choice and guide them appropriately.
5. Keep the tone conversational, concise, and encouraging.

Example greeting:
"Hello! 👋 I'm your TutorAgent assistant. I'm here to help you learn new coding topics, test your knowledge with quizzes, and reinforce what you've learned. What would you like to explore today?"
    """,
    output_key="topics"
)

In [8]:
# Learn Agent: Its job is to teach a user  a new coding topic
learn_agent = Agent(
    name="learn_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    
    instruction="""You are the LearnAgent — a clear, patient, and interactive teaching assistant.
Your role:
1. Receive a topic from the {topics} and explain it in simple, structured language.
2. Explain it in 10-20 lines. In a structured way
3. Ask the user if they would like a deeper or simpler explanation.
4. Do not quiz or test the user — just teach and confirm understanding from the user about the topic
5. When the user finishes learning, notify the controller or routing system that the next step should go to 'quiz_agent'.
6. And store the topic 
""",
output_key = "quiz_topic"
    
)

In [9]:
#quiz Agent: Its job is to teach a user  a new coding topic
quiz_agent = Agent(
    name="quiz_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    
    instruction="""You are the QuizAgent — a helpful quiz master that tests the user’s understanding of a topic.
Your role:
1. Ask 2-3 multiple-choice or short-answer questions based on the {quiz_topic} taught by LearnAgent.
2. Take input from user for the given question and validate whether given topic was understood by user.
3. Give immediate feedback after each question — explain the correct answer briefly.
4. Keep the tone encouraging and educational.
5. When the quiz is complete, summarize the user’s performance.
6. After finishing, signal to the system that the next step should go to 'learn_back_agent' for reflection.
""",
output_key = "teach_back_topic"
    tools = [google_search]
    
)

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3389436998.py, line 18)

In [ ]:
teach_back_agent = Agent(
    name = 'teach_back_agent',
    model = Gemini(
        model = 'gemini-1.5-flash',
        retry_options = retry_config
        
    ),
    instruction= """You are learn_back_agent: ask the user to explain the topic in their own words in 2-3 sentences.
Then provide a short feedback: if the user's response mentions key concepts (look for topic keywords),
say 'Good' and provide 2 corrective pointers if needed.
Output must be:
LEARN_BACK_PROMPT: <one-line prompt asking user to summarize>
(when user provides response, output FEEDBACK: <one-line feedback>)
Finally end with LEARN_BACK_DONE.
""",
    output_key = "learn_back_output"
)

In [ ]:
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[startup_agent,learn_agent, quiz_agent,teach_back_agent],
)

In [ ]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    ""
)

In [ ]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

In [ ]:
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --url_prefix {url_prefix}